In [31]:
import json
import pandas as pd
import glob
import importlib
import datetime
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import datetime
import os
import joblib
import timedelta
import sys
import re
import time

In [32]:
# Supporting 

#Queue
queue_list = pd.read_excel('D:/Code/Code/Support_dashboard.xlsx',sheet_name='Queue List').values
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers

#Alternation
alternation = pd.read_excel('D:/Code/Code/Support_dashboard.xlsx',sheet_name='Alternation')
alternation = alternation[alternation['LOB']=='MOD']
alternation = alternation.iloc[:,[1,2,3,6,9,10,]]

#Week
week_list = pd.read_excel('D:/Code/Code/Support_dashboard.xlsx',sheet_name='Week')


#Diff case / False case - Product comment report
all_sheets_PCR = joblib.load('D:/Code/Joblib_Temp/all_sheets_PCR.pkl')
Diff_case_PCR = all_sheets_PCR['Diff_case_PCR']
False_case_PCR = all_sheets_PCR['False_case_PCR']

#Diff case - Regular
regular = joblib.load('D:/Code/Joblib_Temp/regular diff temp.pkl')
regular_diff = regular['Diff_case_Regular']
regular_false= regular['False_case_Regular']

#Variable
from datetime import datetime
Date_variable = joblib.load('D:/Code/Joblib_Temp/date_df.pkl')

Start_Date_Daily_Report_People_Statistic = datetime.strptime(str(Date_variable.loc['rock_dailyreport', 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_Daily_Report_People_Statistic = datetime.strptime(str(Date_variable.loc['rock_dailyreport', 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

In [33]:
def convert_datetime(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y").strftime(my_format)
def convert_date(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
    
def input_data(data_dir):
    list_files = []
    for filename in pathlib.Path(data_dir).rglob('*.xlsx'): 
        file_suffixes = filename.suffixes
        if file_suffixes[-1].lower() == '.xlsx':
            export_time = os.path.getmtime(filename)
            export_time_datetime = datetime.datetime.fromtimestamp(export_time).strftime('%Y-%m-%d %H:%M:%S')
            dfs = pd.read_excel(filename, sheet_name=None)
            for sheet_name, df in dfs.items():
                df = df.dropna(how='all')
                df['Export time'] = export_time_datetime
                list_files.append(df)
    return pd.concat(list_files, axis=0, ignore_index=True)

def rename_files(data_dir):
    for filename in pathlib.Path(data_dir).glob('*.xlsx'):
        if "People daily d" in filename.name:
            continue
        with pd.ExcelFile(filename) as xlsx:
            df = pd.read_excel(xlsx, sheet_name=None, skiprows=0)
        sheet_name = next(iter(df))
        sheet = df[sheet_name]
        date_str = sheet.at[1, 'Selected duration'].split('~')[0].strip()[:10]
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d').strftime('%Y-%m-%d')
        new_file_name = "People daily d " + date + ".xlsx"
        new_filepath = os.path.join(filename.parent, new_file_name)
        
        i = 1
        while os.path.exists(new_filepath):
            os.remove(new_filepath)
            
            new_file_name = f"People daily d {date}.xlsx"
            new_filepath = os.path.join(filename.parent, new_file_name)
            i += 1
        
        os.rename(filename, new_filepath)

try:
    rename_files('D:/People Statistic/People Daily')
except:
    pass


In [34]:
import datetime
People_statistic_append = input_data('D:/People Statistic/People Daily')
People_statistic_append= People_statistic_append[People_statistic_append['People']!="汇总/All"]
People_statistic_append['Export time'].sort_values(ascending=True)
People_statistic_append = People_statistic_append.drop_duplicates(subset=['Selected duration', 'Claimant queue', 'Sampling Queue', 'People','No. of Samples'],keep='last')

In [35]:
People_statistic_append.columns

Index(['Selected duration', 'Claimant queue', 'Sampling Queue', 'People',
       'No. of Samples', 'No. of Diff', 'badcase', 'Mis-judge rate',
       'Accuracy rate before correction(Related to appeal configuration)',
       'Accuracy rate(Related to appeal configuration)', 'Edge case rate',
       'Appeal amount', 'Appeal rate', 'Successful appeal amount',
       'Successful appeal rate', 'Failed appeal amount', 'Failed appeal rate',
       'Appeal Edge Case', 'Vage case rate', 'Both wrong amount',
       'Both wrong rate', 'Expired', 'Not appeal', 'Handling rate',
       'To be processed amount', 'To be handled rate', 'Sampling rounds',
       'Export time'],
      dtype='object')

In [36]:
People_Round = People_statistic_append[['Selected duration',
                                      'Sampling Queue',
                                      'People',
                                      'No. of Samples']]
People_Round.rename(columns={'People':'Moderator name'},inplace=True)

C:\Users\v6210227\AppData\Local\Temp\ipykernel_19252\368281444.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  People_Round.rename(columns={'People':'Moderator name'},inplace=True)


In [37]:
#Split duration
People_Round[['Moderation time','End date']] = People_Round['Selected duration'].str.split('~', expand=True, n=1)
People_Round['Moderation time'] = pd.to_datetime(People_Round['Moderation time'], format="%Y-%m-%d %H:%M:%S" ).dt.date #%H:%M:%S
People_Round['End date'] = pd.to_datetime(People_Round['End date'], format="%Y-%m-%d %H:%M:%S" ).dt.date #%H:%M:%

C:\Users\v6210227\AppData\Local\Temp\ipykernel_19252\744541444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  People_Round[['Moderation time','End date']] = People_Round['Selected duration'].str.split('~', expand=True, n=1)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_19252\744541444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  People_Round[['Moderation time','End date']] = People_Round['Selected duration'].str.split('~', expand=True, n=1)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_19252\7

# Sample size - Regular

In [38]:
#Sample size
sample_size_filter = People_Round[['Moderation time','Sampling Queue','Moderator name','No. of Samples']]
sample_size_filter['Moderation time'] = pd.to_datetime(sample_size_filter['Moderation time'])

def filter_time( df, from_date, to_date):

    sample_size_filter = df[(df['Moderation time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['Moderation time'].dt.strftime('%Y-%m-%d') <= to_date)]
    sample_size_filter = sample_size_filter.groupby(by=['Moderation time','Sampling Queue','Moderator name'], as_index=False).agg({'No. of Samples':'sum'})
    sample_size_filter = sample_size_filter.drop_duplicates(subset=['Moderation time', 'Sampling Queue', 'Moderator name'],keep='first')
    sample_size_filter = sample_size_filter[['Moderation time','Sampling Queue','Moderator name','No. of Samples']]

    return sample_size_filter, from_date, to_date

# Fill in moderation time
sample_size_filter,from_date, to_date = filter_time(sample_size_filter, Start_Date_Daily_Report_People_Statistic,End_Date_Daily_Report_People_Statistic)

C:\Users\v6210227\AppData\Local\Temp\ipykernel_19252\2300857025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_size_filter['Moderation time'] = pd.to_datetime(sample_size_filter['Moderation time'])


# Diff cases - Regular

In [39]:
False_Rock = pd.concat([regular_false,False_case_PCR],axis=0)

# Daily performance - Regular

In [40]:
import timedelta

daily_performance = pd.merge(sample_size_filter,False_Rock,how='left',left_on=['Moderation time','Sampling Queue','Moderator name'],right_on=['Moderation time','Sampling Queue','Moderator name'])
daily_performance = daily_performance.drop_duplicates(subset=['Moderation time', 'Sampling Queue', 'Moderator name'],keep='first')
daily_performance['No. Diff cases'].fillna(0, inplace=True)
daily_performance = daily_performance[['Moderation time','Sampling Queue','Moderator name','No. of Samples','No. Diff cases']]
lst = ['Sample size','Diff case']

lst[0] = sum(daily_performance['No. Diff cases'])
lst[1] = sum(daily_performance['No. of Samples'])
lst

[24.0, 2955]

In [41]:
daily_performance

,Moderation time,Sampling Queue,Moderator name,No. of Samples,No. Diff cases
0,2023-08-12,QA VN CB Shop Decoration Risk Review,linh.dp4@trans-cosmos.com.vn,1,0.0
1,2023-08-12,QA VN CB Shop Decoration Risk Review,quyen.tt4@trans-cosmos.com.vn,1,0.0
2,2023-08-12,QA VN CB Shop Decoration Risk Review,tuyet.nth@trans-cosmos.com.vn,1,1.0
3,2023-08-12,QA VN CB Shop Decoration Risk Review,yen.tln@trans-cosmos.com.vn,2,1.0
4,2023-08-12,QA VN CB Trademark New,long.dm@trans-cosmos.com.vn,4,0.0
...,...,...,...,...,...
900,2023-08-12,[QA]VN LL Seller On Boarding,trang.ntn1@trans-cosmos.com.vn,11,0.0
901,2023-08-12,[QA]VN LL Seller On Boarding,uyen.hnp1@trans-cosmos.com.vn,3,0.0
902,2023-08-12,[QA]VN LL Seller On Boarding,vi.llt@trans-cosmos.com.vn,1,0.0
903,2023-08-12,[QA]VN LL Seller On Boarding,vi.nq@trans-cosmos.com.vn,1,0.0


In [42]:
#Daily report
daily_performance[['Moderation time','Sampling Queue','Moderator name','No. of Samples','No. Diff cases']]
daily_report_groupqueue = pd.merge(daily_performance,queue_list,how='left',left_on=['Sampling Queue'],right_on=['QA QUEUE NAME'])
daily_report_groupqueue = daily_report_groupqueue[['Moderation time','MOD QUEUE NAME','No. of Samples','No. Diff cases']]
daily_report_groupqueue = daily_report_groupqueue.groupby(['Moderation time','MOD QUEUE NAME'],as_index=False).agg({'No. of Samples':'sum','No. Diff cases':'sum'})
daily_report_groupqueue['%Accr'] = round(1-(daily_report_groupqueue['No. Diff cases'] / daily_report_groupqueue['No. of Samples']),4)
daily_report_groupqueue

,Moderation time,MOD QUEUE NAME,No. of Samples,No. Diff cases,%Accr
0,2023-08-12,VN CB Product Buffer,16,0.0,1.0000
1,2023-08-12,VN CB Product High VV 02,112,1.0,0.9911
2,2023-08-12,VN CB Product Initial Review,112,0.0,1.0000
3,2023-08-12,VN CB Product Report 02,40,1.0,0.9750
4,2023-08-12,VN CB Shop Decoration Risk Review,5,2.0,0.6000
5,2023-08-12,VN CB Trademark New,24,0.0,1.0000
6,2023-08-12,VN LL Anchor Video 02,284,2.0,0.9930
7,2023-08-12,VN LL Anchor Video Counterfeit Key Frame,111,0.0,1.0000
8,2023-08-12,VN LL Anchor Video High VV 02,110,0.0,1.0000
9,2023-08-12,VN LL Anchor Video Report 02,109,0.0,1.0000


In [43]:
all_sheets_regular = {'Diff_case_Regular':regular_diff,
                      'Diff_case_PCR': Diff_case_PCR,
                      'Daily_Performance_Regular':daily_performance,
                      'Sample_size_Regular':sample_size_filter,
                      'False_case_Regular':False_Rock,
                      'DR_group_queue_Regular': daily_report_groupqueue}

joblib.dump(all_sheets_regular, 'D:/Code/Joblib_Temp/all_sheets_regular.pkl')

['D:/Code/Joblib_Temp/all_sheets_regular.pkl']

In [44]:
# Regular_writer = pd.ExcelWriter('C:/Users/v6210227/Desktop/regular.xlsx', engine='xlsxwriter')

# for sheet_name in all_sheets_regular.keys():
#     all_sheets_regular[sheet_name].to_excel(excel_writer=Regular_writer, sheet_name=sheet_name)
   
# Regular_writer.close()
